In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

Define a custom NMF function:

In [1]:
def nmf(X, W, H, n_iter=100):
    import numpy as np
    for iter in range(n_iter):
        # update H
        H = H * np.divide(W.T @ X, W.T @ W @ H)
        # update W
        W = W * np.divide(X @ H.T, W @ H @ H.T)
    return W, H

In [2]:
#
# Generate Data
#
import sys; sys.path.append("../../scripts/")
from generate_X import gen_data

X = gen_data(R=4, shape=[10,20])["X"]
X.shape

../../scripts/generate_X.py:71: UserWarning: Kruskal's theorem probably won't apply, may not have a unique nCPD.
  warnings.warn("Kruskal's theorem probably won't apply, may not have a unique nCPD.")


(10, 20)

In [3]:
from TELF.factorization import NMFk
params = {
    "n_perturbs":10,
    "n_iters":10,
    "epsilon":0.015,
    "n_jobs":1,
    "init":"nnsvd", 
    "use_gpu":False,
    "save_path":"../../results/",
    "save_output":True,
    "collect_output":True,
    "predict_k":True,
    "verbose":True,
    "transpose":False,
    "sill_thresh":0.9,
    "nmf_verbose":False,
    "nmf_method":"func",
    "nmf_func":nmf # custom function
}
model = NMFk(**params)

Performing NMF with  func


/Users/maksim/opt/miniconda3/envs/TELF/lib/python3.8/site-packages/TELF-0.0.1-py3.8.egg/TELF/factorization/NMFk.py:392: UserWarning: calculate_error is True! Error calculation can make the runtime longer and take up more memory space!
/Users/maksim/opt/miniconda3/envs/TELF/lib/python3.8/site-packages/TELF-0.0.1-py3.8.egg/TELF/factorization/NMFk.py:396: UserWarning: predict_k is True with pvalue method! Predicting k can make the runtime significantly longer. Consider using predict_k_method='sill'.


In [4]:
Ks = range(2,6,2)
name = "Example"
note = "This is an example run of NMFk"
results = model.fit(X, Ks, name, note)
print(results.keys())

100%|██████████| 2/2 [00:00<00:00, 31.09it/s]


dict_keys(['time', 'k_predict', 'W', 'H'])
